In [ ]:
# !pip install -q /kaggle/input/cbmrdataset/ecos-2.0.14-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install -q /kaggle/input/cbmrdataset/joblib-1.4.0-py3-none-any.whl
# !pip install -q /kaggle/input/cbmrdataset/joblib-1.4.2-py3-none-any.whl
# !pip install -q /kaggle/input/cbmrdataset/numexpr-2.10.2-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl
# !pip install -q /kaggle/input/cbmrdataset/numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install -q /kaggle/input/cbmrdataset/numpy-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install -q /kaggle/input/cbmrdataset/osqp-0.6.7.post3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install -q /kaggle/input/cbmrdataset/pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install -q /kaggle/input/cbmrdataset/python_dateutil-2.9.0.post0-py2.py3-none-any.whl
# !pip install -q /kaggle/input/cbmrdataset/pytz-2024.2-py2.py3-none-any.whl
# !pip install -q /kaggle/input/cbmrdataset/qdldl-0.1.7.post4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install -q /kaggle/input/cbmrdataset/scikit_learn-1.4.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install -q /kaggle/input/cbmrdataset/scikit_learn-1.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install -q /kaggle/input/cbmrdataset/scikit_survival-0.23.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install -q /kaggle/input/cbmrdataset/scipy-1.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install -q /kaggle/input/cbmrdataset/scipy-1.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install -q /kaggle/input/cbmrdataset/six-1.17.0-py2.py3-none-any.whl
# !pip install -q /kaggle/input/cbmrdataset/threadpoolctl-3.4.0-py3-none-any.whl
# !pip install -q /kaggle/input/cbmrdataset/threadpoolctl-3.5.0-py3-none-any.whl
# !pip install -q /kaggle/input/cbmrdataset/tzdata-2024.2-py2.py3-none-any.whl
# #

In [ ]:
# pip install scikit-learn==1.4.2
# pip install numpy==1.26.4
# pip install threadpoolctl==3.4.0
# pip install joblib==1.4.0
# pip install scipy==1.13.0
# pip install scikit-survival

In [2]:
pip install --no-index --find-links=/kaggle/input/cmbrdata scikit-survival


Looking in links: /kaggle/input/cmbrdata
Processing /kaggle/input/cmbrdata/scikit_survival-0.23.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/cmbrdata/scikit_learn-1.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (from scikit-survival)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import time

from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sksurv.metrics import concordance_index_censored
from sksurv.util import Surv

from sklearn.model_selection import cross_val_score


In [4]:
train_data = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/train.csv")

In [27]:
train_data.shape

(28800, 59)

In [5]:
gc.collect()

20

In [6]:
df = train_data.drop(columns=['ID'])

In [7]:
del train_data
gc.collect()

0

In [8]:
def handle_columns(df):
    """
    Handles both categorical and numerical columns in a DataFrame.
    
    - Categorical columns:
        - Replaces NaN with a placeholder 'NaN' to distinguish missing values.
        - Applies label encoding specific to each column.
        - Ensures the dtype remains 'category' after encoding.
    
    - Numerical columns:
        - Replaces NaN values with the column mean.
        - Downcasts float64 to float32 and int64 to int32 for memory efficiency.

    Parameters:
    - df (pd.DataFrame): Input DataFrame to process.

    Returns:
    - pd.DataFrame: Processed DataFrame with handled missing values and optimized data types.
    """
    df = df.copy() 

    # Handle Categorical Columns
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    
    for col in categorical_cols:
        # Replace NaN with 'Missing' for categorical columns
        df[col] = df[col].fillna('NaN')
        
        # Apply Label Encoding to each categorical column
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        
        # Convert back to 'category' dtype
        df[col] = df[col].astype('category')
    
    # Handle Numerical Columns
    numerical_cols = df.select_dtypes(include=['number']).columns
    
    for col in numerical_cols:
        # Replace NaN values with the column mean
        df[col] = df[col].fillna(df[col].mean())
        
        # Downcast to reduce memory usage
        if df[col].dtype == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')  # Downcast to float32
        elif df[col].dtype == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')  # Downcast to int32
    
    return df


In [9]:
train_data = handle_columns(df)

In [28]:
train_data

,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,...,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time
0,7,1,2,1,1.764516,6.876801,0,1,6.0,0,...,3,3,1,8.0,1,2.0,1,10.0,False,42.355999
1,2,1,1,1,2.000000,8.000000,6,1,6.0,1,...,4,2,1,8.0,1,2.0,3,10.0,True,4.672000
2,7,1,2,1,2.000000,8.000000,0,1,6.0,0,...,4,2,1,8.0,1,2.0,1,10.0,False,19.792999
3,0,1,1,1,2.000000,8.000000,0,1,6.0,0,...,4,3,1,8.0,1,2.0,1,10.0,False,102.348999
4,0,1,2,1,2.000000,8.000000,0,1,6.0,1,...,4,2,0,8.0,1,2.0,1,10.0,False,16.223000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28795,3,0,0,1,2.000000,8.000000,0,1,6.0,1,...,0,1,1,8.0,0,2.0,1,10.0,False,18.632999
28796,0,1,6,3,1.000000,4.000000,0,1,5.0,1,...,1,2,1,6.0,3,1.0,3,8.0,True,4.892000
28797,10,0,6,0,2.000000,8.000000,0,0,6.0,1,...,1,3,1,8.0,0,2.0,1,10.0,False,23.157000
28798,7,1,6,1,1.000000,4.000000,0,1,3.0,1,...,4,2,0,4.0,1,1.0,1,5.0,False,52.351002


In [10]:
del df

In [11]:
gc.collect()

0

In [12]:
if 'efs' not in train_data.columns or 'efs_time' not in train_data.columns:
    raise ValueError("The dataset must contain 'efs' (event) and 'efs_time' (time-to-event) columns.")

# Convert 'efs' to boolean for survival analysis
train_data['efs'] = train_data['efs'].astype(bool)

survival_data = Surv.from_dataframe("efs", "efs_time", train_data)

X = train_data.drop(columns=['efs', 'efs_time'])  # Drop non-feature columns like 'ID'
y = survival_data

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
y

array([(False, 42.35599899), ( True,  4.67199993), (False, 19.79299927),
       ..., (False, 23.15699959), (False, 52.35100174),
       (False, 25.15800095)], dtype=[('efs', '?'), ('efs_time', '<f8')])

In [17]:
gc.collect()

26

In [18]:
rsf = RandomSurvivalForest(
    n_estimators=38, max_depth=11,
    min_samples_split=6, min_samples_leaf=10, n_jobs=-1, random_state=31,max_features='log2'
)
print("Training started...")

start = time.time()
rsf.fit(X_train, y_train)
end = time.time()

print(f"Total time to Train was {end-start}")

Training started...
Total time to Train was 70.96837067604065


In [19]:
gc.collect()

28

In [20]:
test_df = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")
test_data = handle_columns(test_df)

In [1]:
risk_scores = rsf.predict(X_val)

# Evaluate the model using Concordance Index (C-index)
c_index = concordance_index_censored(
    event_indicator=y_val['efs'],
    event_time=y_val['efs_time'],
    estimate=risk_scores
)

print("Concordance Index (C-index):", c_index[0])

NameError: name 'rsf' is not defined

In [30]:
risk_scores

array([5038.31562022, 2158.31039027, 4203.54854724, ...,  573.10177146,
       3075.44005988, 3015.03486836])

In [22]:
# ====================================
test_data_ = test_data.drop(columns=['ID'])
test_risk_scores = rsf.predict(test_data_)

# Prepare submission file (without 'ID' column)
submission = pd.DataFrame({
    "ID": test_data["ID"],
    "prediction": test_risk_scores   
})

# Save the submission file
submission.to_csv("submission.csv", index=False)
print("Submission file created: submission.csv")

Submission file created: submission.csv


In [23]:
submission

,ID,prediction
0,28800,2277.376570
1,28801,4082.315331
2,28802,2380.304487
